In [6]:
import google.cloud.bigquery as bq
import pandas as pd

In [2]:
%load_ext google.cloud.bigquery

In [3]:
%%bigquery stations

SELECT
stat.id,
LEFT(stat.id, 2) AS country,
stat.state,
stat.name,
stat.latitude,
stat.longitude,
inv.element,
inv.firstyear,
inv.lastyear 
FROM `bigquery-public-data.ghcn_d.ghcnd_stations` stat
LEFT JOIN `bigquery-public-data.ghcn_d.ghcnd_inventory` inv ON stat.id = inv.id
  AND inv.element IN ("TMAX", "TMIN", "ACMC", "AWDR", "AWND", "EVAP", "MXPN")
WHERE
ST_CONTAINS(
  ST_MAKEPOLYGON(
    ST_MAKELINE(
      ARRAY(
              select ST_GEOGPOINT(-12.950869, 53.431059) union all
              select ST_GEOGPOINT(-10.342842, 35.002107) union all
              select ST_GEOGPOINT(-19.931662, 13.986127) union all
              select ST_GEOGPOINT(-9.352620, 0.0)        union all
              select ST_GEOGPOINT(-49.101915, 0.384726)  union all
              select ST_GEOGPOINT(-71.903541, 13.516013) union all
              select ST_GEOGPOINT(-76.957330, 34.883146)
            )
          )
        )
  ,ST_GEOGPOINT(stat.longitude, stat.latitude)) IS TRUE
ORDER BY stat.id

In [4]:
stations.head()

,id,country,state,name,latitude,longitude,element,firstyear,lastyear
0,ACW00011647,AC,,ST JOHNS,17.1333,-61.7833,TMAX,1961.0,1961.0
1,ACW00011647,AC,,ST JOHNS,17.1333,-61.7833,TMIN,1961.0,1961.0
2,BB000078954,BB,,GRANTLEY ADAMS,13.0670,-59.4830,TMAX,1944.0,2018.0
3,BB000078954,BB,,GRANTLEY ADAMS,13.0670,-59.4830,TMIN,1944.0,2018.0
4,BDM00078016,BD,,L F WADE INTL AP KINDLEY FLD,32.3667,-64.6833,TMAX,1949.0,2020.0


In [13]:
stations_pivot = pd.pivot_table(data=stations,
                                index=['country', 'id'],
                                columns='element',
                                values=['firstyear', 'lastyear'])

stations_pivot['n_missing'] = stations_pivot.isnull().sum(axis=1)

In [16]:
stations_pivot.sort_values('n_missing').head(10)

firstyear                                 lastyear  \
element                  AWND    EVAP    MXPN    TMAX    TMIN     AWND   
country id                                                               
RQ      RQW00011631       NaN  1916.0  1963.0  1916.0  1916.0      NaN   
        RQC00664276       NaN  1961.0  1968.0  1956.0  1956.0      NaN   
        RQC00664702       NaN  1963.0  1970.0  1901.0  1901.0      NaN   
        RQC00660061       NaN  1970.0  1981.0  1970.0  1970.0      NaN   
        RQC00665097       NaN  1956.0  1977.0  1900.0  1900.0      NaN   
        RQC00667292       NaN  1969.0  1969.0  1898.0  1898.0      NaN   
        RQC00668306       NaN  1959.0  2000.0  1959.0  1959.0      NaN   
        RQC00669829       NaN  1971.0  1971.0  1971.0  1971.0      NaN   
VQ      VQC00670480       NaN  1972.0  1972.0  1972.0  1972.0      NaN   
RQ      RQC00662934       NaN  1964.0  1964.0  1931.0  1931.0      NaN   

                                                    n_missing  
element                EVAP    MXPN    TMAX    TMIN            
country id                                                     
RQ      RQW00011631  1967.0  1967.0  1977.0  1977.0         2  
        RQC00664276  2015.0  2013.0  2015.0  2015.0         2  
        RQC00664702  2020.0  2018.0  2020.0  2020.0         2  
        RQC00660061  2018.0  2018.0  2020.0  2020.0         2  
        RQC00665097  2019.0  2016.0  2020.0  2020.0         2  
        RQC00667292  2020.0  2010.0  2020.0  2020.0         2  
        RQC00668306  2017.0  2017.0  2020.0  2020.0         2  
        RQC00669829  1995.0  1995.0  1995.0  1995.0         2  
VQ      VQC00670480  2014.0  2014.0  2014.0  2014.0         2  
RQ      RQC00662934  2012.0  2013.0  2013.0  2013.0         2